# American Sign Language

The American Sign Language letter database of hand gestures represent a multi-class problem with 24 classes of letters (excluding J and Z which require motion).

The dataset format is patterned to match closely with the classic MNIST. Each training and test case represents a label (0-25) as a one-to-one map for each alphabetic letter A-Z (and no cases for 9=J or 25=Z because of gesture motions). The training data (27,455 cases) and test data (7172 cases) are approximately half the size of the standard MNIST but otherwise similar with a header row of label, pixel1,pixel2….pixel784 which represent a single 28x28 pixel image with grayscale values between 0-255. The original hand gesture image data represented multiple users repeating the gesture against different backgrounds.

![](https://d.newsweek.com/en/full/1394686/asl-getty-images.jpg)

If you found any part of this notebook useful, please leave an **UPVOTE!!**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelBinarizer
from keras.callbacks import ReduceLROnPlateau
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.preprocessing import image
from matplotlib.pyplot import imshow
from keras.applications.imagenet_utils import preprocess_input

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

Reading data

In [ ]:
train = pd.read_csv("../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv")
test = pd.read_csv("../input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv")

In [ ]:
train.head(10)

In [ ]:
test.head(10)

Separating dependent and independent variables

In [ ]:
x_train = train.drop(labels = ["label"],axis = 1) 
y_train = train["label"]

In [ ]:
x_train.head()

In [ ]:
y_train.head()

Let's see the frequency distribution for each label

In [ ]:
y_train.value_counts().plot.bar()

In [ ]:
x_test = test.drop(labels = ["label"],axis = 1) 
y_test = test["label"]

In [ ]:
x_train = train.drop(labels = ["label"],axis = 1) 
y_train = train["label"]

In [ ]:
x_train.head()

Normalize data set to 0 to 1 range. This helps the optimizer to converge faster as compared to 0-255 range

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [ ]:
x_train

Reshaping to the format(3D)that is accepted by CNN model

In [ ]:
x_train = x_train.values.reshape(-1,28,28,1)
x_test = x_test.values.reshape(-1,28,28,1)

Lets see the images

In [ ]:
plt.imshow(x_train[1].reshape(28,28))

Data augmentation is a technique which allows us to create more samples artifically. New images are created by distorting original images slightly. For example, taking random crop of the image or rotating it or flipping it sideways.

In [ ]:
dataAug = ImageDataGenerator(
        rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False)  # randomly flip images

dataAug.fit(x_train)

In [ ]:
x_train.shape

One-hot-Encoding the dependent variable

In [ ]:
label_binarizer = LabelBinarizer()
y_train = label_binarizer.fit_transform(y_train)
y_test = label_binarizer.fit_transform(y_test)

Encoded classes

In [ ]:
label_binarizer.classes_

In [ ]:
class_names = ["A","B","C","D","E","F","G","H","I","K","L",'M','N','O','P','Q','R','S','T','U','V','W','X','Y']

In [ ]:
y_train.shape

Splitting train data into train and validation

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, random_state=42)

Reduce learning rate when a metric (val_accuracy) has stopped improving.

In [ ]:
lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.5, min_lr=0.00001)

# Designing the model

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding = "same", input_shape=(28,28,1), activation="relu"))
model.add(Conv2D(32, (3, 3), padding = "same", input_shape=(28,28,1), activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),strides = (2,2)))


model.add(Conv2D(64, (3, 3),padding = "same", activation="relu"))
model.add(Conv2D(64, (3, 3),padding = "same", activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Conv2D(128, (3, 3),padding = "same", activation="relu"))
model.add(Conv2D(128, (3, 3),padding = "same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3),strides = (2,2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(24, activation="softmax"))

# Model Summary

In [ ]:
model.summary()

In [ ]:
plot_model(model, to_file='Model.png')
SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer="adam",
    metrics=['accuracy']
)

In [ ]:
model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=30,
    validation_data=(x_val, y_val),
    shuffle=True,
    callbacks = [lr_reduce]
)


Evaluation on test set

In [ ]:
preds = model.evaluate(x_test,y_test)

print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

# Try prediction on your own image!

In [ ]:
img_path = '../input/myhand/WIN_20200918_18_00_39_Pro (2).jpg'
img = image.load_img(img_path, target_size=(28, 28),  color_mode="grayscale")
imshow(img,cmap = "gray")
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
#x = x/255
x = np.vstack([x])
print("Predicted letter is: "+ class_names[np.argmax(model.predict(x))])

Checking on Test data

In [ ]:
preds = model.predict(x_test)

In [ ]:
n=5
plt.imshow(x_test[n].reshape(28,28),cmap="gray") 
plt.grid(False) 
print("Predicted letter is:",class_names[np.argmax(preds[n])],"\nActual Answer:",class_names[np.argmax(y_test[n])]) # Prediction - True Answer